In [47]:
import pandas as pd
import numpy as np

In [48]:
df_merged = pd.read_csv('/Users/tranminu/Documents/Fertility-Rate-Prediction/data/clean/final_merged_dataset_v2.csv')
display(df_merged.head(5))

,Country Name,Country Code,Year,Female share of employment in senior and middle management (%),"Maternal mortality ratio (per 100,000 live births)","Labor force participation rate, female (%)",advanced_education_pct,basic_education_pct,Fertirity rate,Urban population over total population,Capitial GDP in USD,Access to electricity (% of population),WagedFemale,InfantMortality,LifeExpectancy
0,Afghanistan,AFG,1960.0,NaN,NaN,NaN,NaN,NaN,7.282,8.401,NaN,NaN,NaN,251.2,32.799
1,Afghanistan,AFG,1961.0,NaN,NaN,NaN,NaN,NaN,7.284,8.684,NaN,NaN,NaN,248.4,33.291
2,Afghanistan,AFG,1962.0,NaN,NaN,NaN,NaN,NaN,7.292,8.976,NaN,NaN,NaN,245.4,33.757
3,Afghanistan,AFG,1963.0,NaN,NaN,NaN,NaN,NaN,7.302,9.276,NaN,NaN,NaN,242.5,34.201
4,Afghanistan,AFG,1964.0,NaN,NaN,NaN,NaN,NaN,7.304,9.586,NaN,NaN,NaN,239.7,34.673


**I) Data cleaning and missing value duel**

In [ ]:
drop_countries = [
    'AFE', 'AFW', 'ARB', 'CEB', 'CSS', 'EAP', 'EAR', 'EAS', 'ECA', 'ECS', 
    'EMU', 'EUU', 'FCS', 'HIC', 'HPC', 'IBD', 'IBT', 'IDA', 'IDB', 'IDX',
    'INX', 'LAC', 'LCN', 'LDC', 'LIC', 'LMC', 'LMY', 'LTE', 'MEA', 'MIC',
    'MNA', 'NAC', 'OED', 'OSS', 'PRE', 'PSS', 'PST', 'SAS', 'SSA', 'SSF',
    'SST', 'TEA', 'TEC', 'TLA', 'TMN', 'TSA', 'TSS', 'UMC', 'WLD'
] 
df_merged_edited = df_merged[~df_merged['Country Code'].isin(drop_countries)]

print(f"Null values for each column:")
null_values = df_merged_edited.isna().sum().sort_values(ascending=False) / df_merged_edited.shape[0]
display(null_values)
drop_columns = ['Female share of employment in senior and middle management (%)',
       'advanced_education_pct', 'basic_education_pct',
       'Labor force participation rate, female (%)',
       'Access to electricity (% of population)',
       'Maternal mortality ratio (per 100,000 live births)', 'WagedFemale']

if drop_columns in df_merged_edited.columns:
    df_merged_edited = df_merged_edited.drop(columns= drop_columns)

display(df_merged_edited.head(5))

Null values for each column:


Female share of employment in senior and middle management (%)    0.911077
advanced_education_pct                                            0.877804
basic_education_pct                                               0.876999
Labor force participation rate, female (%)                        0.751959
Access to electricity (% of population)                           0.649458
Maternal mortality ratio (per 100,000 live births)                0.593968
WagedFemale                                                       0.553075
Capitial GDP in USD                                               0.390040
InfantMortality                                                   0.301009
Urban population over total population                            0.261565
Fertirity rate                                                    0.256413
Country Code                                                      0.254696
LifeExpectancy                                                    0.095954
Year                     

TypeError: unhashable type: 'list'

In [ ]:
df_corr = df_merged_edited.drop(columns =['Country Name', 'Country Code'],inplace = False)
df_corr = df_merged_edited.select_dtypes(include=['number'])

df_corr.head(5)
print(df_corr.corr()[['Fertirity rate']].sort_values(by= 'Fertirity rate', ascending= False))

                                        Fertirity rate
Fertirity rate                                1.000000
InfantMortality                               0.830195
Capitial GDP in USD                          -0.435065
Year                                         -0.482118
Urban population over total population       -0.613451
LifeExpectancy                               -0.844801
